## import required libraries

In [23]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## Load Datasets

In [2]:
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

C:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Check for null values

In [42]:
print("Books DF Null value-------------------------------")
print(books.isnull().sum(),"\n")
print("User DF Null Value-------------------------------")
print(users.isnull().sum(),"\n")
print("Ratings DF Null Value-------------------------------")
print(ratings.isnull().sum())

Books DF Null value-------------------------------
ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64 

User DF Null Value-------------------------------
User-ID          0
Location         0
Age         110762
dtype: int64 

Ratings DF Null Value-------------------------------
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


## Check for duplicates

In [46]:
print("Books DF duplicate value-------------------------------")
print(books.duplicated().sum(),"\n")
print("user DF duplicate value-------------------------------")
print(users.duplicated().sum(),"\n")
print("ratings DF duplicate value-------------------------------")
print(ratings.duplicated().sum())

Books DF duplicate value-------------------------------
0 

user DF duplicate value-------------------------------
0 

ratings DF duplicate value-------------------------------
0


## Popularity based recommendation 

In [3]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [4]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [5]:
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [6]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [7]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [8]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [12]:
popular_df['Book-Title'][0]

'Harry Potter and the Prisoner of Azkaban (Book 3)'

# Collaborative Filtering Based Recommender System

In [14]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
frequent_user = x[x].index

In [16]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(frequent_user)]

In [17]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
popular_books = y[y].index

In [18]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(popular_books)]

In [19]:
df = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
df.head(3)

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN


In [22]:
df.fillna(0,inplace=True)
df.head(3)

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
similarity_scores = cosine_similarity(df)

In [31]:
def recommend(book_name):
    #  fetch index
    index = np.where(df.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:3]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == df.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [32]:
recommend('2nd Chance')

[['Four Blind Mice',
  'James Patterson',
  'http://images.amazon.com/images/P/0316693006.01.MZZZZZZZ.jpg'],
 ['The Next Accident',
  'LISA GARDNER',
  'http://images.amazon.com/images/P/0553578693.01.MZZZZZZZ.jpg']]